<a href="https://colab.research.google.com/github/16A0/til/blob/master/Aphantasia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  # from pprint import pprint
  translator = Translator()
except: pass
!pip install ftfy==5.8

import os
import io
import time
from math import exp
import random
import imageio
import numpy as np
import PIL
from skimage import exposure
from base64 import b64encode
import moviepy, moviepy.editor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

from IPython.display import HTML, Image, display, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import ipywidgets as ipy
# import glob
from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/openai/CLIP.git
import clip
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
!pip install git+https://github.com/Po-Hsun-Su/pytorch-ssim
import pytorch_ssim as ssim

%cd /content
!rm -rf aphantasia
!git clone https://github.com/eps696/aphantasia
%cd aphantasia/
from clip_fft import to_valid_rgb, fft_image, slice_imgs, checkout
from utils import pad_up_to, basename, img_list, img_read
from progress_bar import ProgressIPy as ProgressBar

workdir = '_out'
tempdir = os.path.join(workdir, 'ttt')

clear_output()

resume = True #@param {type:"boolean"}
if resume:
  resumed = files.upload()
  params_pt = list(resumed.values())[0]
  params_pt = torch.load(io.BytesIO(params_pt))

def makevid(seq_dir, size=None):
  # out_sequence = seq_dir + '/%05d.jpg'
  out_video = seq_dir + '.mp4'
  # !ffmpeg -y -v quiet -i $out_sequence $out_video
  moviepy.editor.ImageSequenceClip(img_list(seq_dir), fps=25).write_videofile(out_video, verbose=False) # , ffmpeg_params=ffmpeg_params, logger=None
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

!nvidia-smi -L
print('\nDone!')

MessageError: ignored

Source https://github.com/eps696/aphantasia/blob/master/Aphantasia.ipynb


Type some text and/or upload some image to start.
fine_details input would make micro details follow that topic.
Put to subtract the topics, which you would like to avoid in the result.
NB: more prompts = more memory! (handled by auto-decreasing samples amount, hopefully you don't need to act).
invert the whole criteria, if you want to see "the totally opposite".

Options for non-English languages (use only one of them!):
multilang = use multi-language model, trained with ViT
translate = use Google translate (works with any visual model)

In [ ]:
#@title Input

text = "sunrise on town at the edge of ocean cliff" #@param {type:"string"}
fine_details = "scottish highland " #@param {type:"string"}
subtract = "" #@param {type:"string"}
multilang = False #@param {type:"boolean"}
translate = False #@param {type:"boolean"}
invert = False #@param {type:"boolean"}
upload_image = False #@param {type:"boolean"}

if translate:
  text = translator.translate(text, dest='en').text
if upload_image:
  uploaded = files.upload()

Settings
Select CLIP visual model (results do vary!). I prefer ViT for consistency (and it's the only native multi-language option).
overscan option produces semi-seamlessly tileable texture (when off, it's more centered).
sync value adds SSIM loss between the output and input image (if there's one), allowing to "redraw" it with controlled similarity.

Decrease samples if you face OOM (it's the main RAM eater).
Setting steps much higher (1000-..) will elaborate details and make tones smoother, but may start throwing texts like graffiti.

Other tricks:
diverse boosts compositional & contextual variety (difference between simultaneous samples). good start is ~0.2; can be negative.
expand boosts training steps in general (difference between consequent samples). good start is ~0.2.
progressive_grow may boost macro forms creation (especially with lower learning_rate), see more [here](https://github.com/eps696/aphantasia/issues/2).

In [ ]:
#@title Generate

# from google.colab import drive
# drive.mount('/content/GDrive')
# clipsDir = '/content/GDrive/MyDrive/T2I ' + dtNow.strftime("%Y-%m-%d %H%M")

!rm -rf $tempdir
os.makedirs(tempdir, exist_ok=True)

sideX = 1280 #@param {type:"integer"}
sideY = 720 #@param {type:"integer"}
#@markdown > Config
model = 'ViT-B/32' #@param ['ViT-B/32', 'RN101', 'RN50x4', 'RN50']
overscan = True #@param {type:"boolean"}
sync =  0.5 #@param {type:"number"}
contrast = 1. #@param {type:"number"}
#@markdown > Training
steps = 200 #@param {type:"integer"}
samples = 200 #@param {type:"integer"}
learning_rate = .05 #@param {type:"number"}
save_freq = 1 #@param {type:"integer"}
#@markdown > Tricks
diverse = 0. #@param {type:"number"}
expand = 0. #@param {type:"number"}
progressive_grow = False #@param {type:"boolean"}
if multilang: model = 'ViT-B/32' # sbert model is trained with ViT

if len(fine_details) > 0:
  samples = int(samples * 0.75)
if len(subtract) > 0:
  samples = int(samples * 0.75)
print(' using %d samples' % samples)

model_clip, _ = clip.load(model)
modsize = 288 if model == 'RN50x4' else 224
xmem = {'RN50':0.5, 'RN50x4':0.16, 'RN101':0.33}
if 'RN' in model:
  samples = int(samples * xmem[model])

if multilang is True:
    model_lang = SentenceTransformer('clip-ViT-B-32-multilingual-v1').cuda()

def enc_text(txt):
    if multilang is True:
        emb = model_lang.encode([txt], convert_to_tensor=True, show_progress_bar=False)
    else:
        emb = model_clip.encode_text(clip.tokenize(txt).cuda())
    return emb.detach().clone()

if diverse != 0:
  samples = int(samples * 0.5)
        
norm_in = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
sign = 1. if invert is True else -1.

if upload_image:
  in_img = list(uploaded.values())[0]
  print(' image:', list(uploaded)[0])
  img_in = torch.from_numpy(imageio.imread(in_img).astype(np.float32)/255.).unsqueeze(0).permute(0,3,1,2).cuda()[:,:3,:,:]
  in_sliced = slice_imgs([img_in], samples, modsize, transform=norm_in)[0]
  img_enc = model_clip.encode_image(in_sliced).detach().clone()
  if sync > 0:
    overscan = True
    ssim_loss = ssim.SSIM(window_size = 11)
    ssim_size = [sideY//8, sideX//8]
    img_in = F.interpolate(img_in, ssim_size).float()
    # img_in = F.interpolate(img_in, (sideY, sideX)).float()
  else:
    del img_in
  del in_sliced; torch.cuda.empty_cache()

if len(text) > 2:
  print(' macro:', text)
  if translate:
    translator = Translator()
    text = translator.translate(text, dest='en').text
    print(' translated to:', text) 
  txt_enc = enc_text(text)

if len(fine_details) > 0:
  print(' micro:', fine_details)
  if translate:
      translator = Translator()
      fine_details = translator.translate(fine_details, dest='en').text
      print(' translated to:', fine_details) 
  txt_enc2 = enc_text(fine_details)

if len(subtract) > 0:
  print(' without:', subtract)
  if translate:
      translator = Translator()
      subtract = translator.translate(subtract, dest='en').text
      print(' translated to:', subtract) 
  txt_enc0 = enc_text(subtract)

if multilang is True: del model_lang

shape = [1, 3, sideY, sideX]
param_f = fft_image 
# param_f = pixel_image
# learning_rate = 1.
init_pt = params_pt if resume is True else None
params, image_f = param_f(shape, resume=init_pt)
image_f = to_valid_rgb(image_f)

if progressive_grow is True:
  lr1 = learning_rate * 2
  lr0 = lr1 * 0.01
else:
  lr0 = learning_rate
optimizer = torch.optim.Adam(params, lr0)

def save_img(img, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1,2,0))  
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave('result.jpg', np.array(img))

def checkout(num):
  with torch.no_grad():
    img = image_f(contrast=contrast).cpu().numpy()[0]
  save_img(img, os.path.join(tempdir, '%04d.jpg' % num))
  outpic.clear_output()
  with outpic:
    display(Image('result.jpg'))

prev_enc = 0
def train(i):
  loss = 0
  img_out = image_f()

  micro = False if len(fine_details) > 0 else None
  imgs_sliced = slice_imgs([img_out], samples, modsize, norm_in, overscan=overscan, micro=micro)
  out_enc = model_clip.encode_image(imgs_sliced[-1])
  if diverse != 0:
    imgs_sliced = slice_imgs([image_f()], samples, modsize, norm_in, overscan=overscan, micro=micro)
    out_enc2 = model_clip.encode_image(imgs_sliced[-1])
    loss += diverse * torch.cosine_similarity(out_enc, out_enc2, dim=-1).mean()
    del out_enc2; torch.cuda.empty_cache()
  if upload_image:
      loss += sign * 0.5 * torch.cosine_similarity(img_enc, out_enc, dim=-1).mean()
  if len(text) > 0: # input text
      loss += sign * torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
  if len(subtract) > 0: # subtract text
      loss += -sign * torch.cosine_similarity(txt_enc0, out_enc, dim=-1).mean()
  if sync > 0 and upload_image: # image composition sync
      loss -= sync * ssim_loss(F.interpolate(img_out, ssim_size).float(), img_in)
  if len(fine_details) > 0: # input text for micro details
      imgs_sliced = slice_imgs([img_out], samples, modsize, norm_in, overscan=overscan, micro=True)
      out_enc2 = model_clip.encode_image(imgs_sliced[-1])
      loss += sign * torch.cosine_similarity(txt_enc2, out_enc2, dim=-1).mean()
      del out_enc2; torch.cuda.empty_cache()
  if expand > 0:
    global prev_enc
    if i > 0:
      loss += expand * torch.cosine_similarity(out_enc, prev_enc, dim=-1).mean()
    prev_enc = out_enc.detach()
  del img_out, imgs_sliced, out_enc; torch.cuda.empty_cache()

  if progressive_grow is True:
    lr_cur = lr0 + (i / steps) * (lr1 - lr0)
    for g in optimizer.param_groups: 
      g['lr'] = lr_cur

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
  if i % save_freq == 0:
    checkout(i // save_freq)

outpic = ipy.Output()
outpic

pbar = ProgressBar(steps)
for i in range(steps):
  train(i)
  _ = pbar.upd()

HTML(makevid(tempdir))
torch.save(params, tempdir + '.pt')
files.download(tempdir + '.pt')
files.download(tempdir + '.mp4')